In [1]:
from crm import Network

In [2]:
import re
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm.notebook import trange

In [3]:
import dill
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        dill.dump(obj, outp, dill.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as inp:
        return dill.load(inp)

# Data

In [4]:
# https://stackoverflow.com/questions/4666973/how-to-extract-the-substring-between-two-markers

In [5]:
f = open("data/sample5/raw/rand_crm_1np.pl")

In [6]:
edges_raw = []

In [7]:
while True:
    s = f.readline()
    if not s:
        break
    if "connected" in s:
        edges_raw.append(s)

In [8]:
edges_raw[24]

'connected(a(953),[a(1),a(69)],[1,1]).\n'

In [9]:
edges_raw[24].split(",")[0].split("(")[-1].split(")")[0]

'953'

In [10]:
gg = re.search("\[a(\(.+?\))\]", edges_raw[-2])

In [11]:
gg.group(1)

'(0),a(0)'

In [12]:
edges_raw[24]

'connected(a(953),[a(1),a(69)],[1,1]).\n'

In [13]:
edges_raw[-2]

'connected(a(2),[a(0),a(0)],[1,0]).\n'

In [14]:
re.search("\[(a.+?)\]", edges_raw[-2]).group(1)

'a(0),a(0)'

In [15]:
re.search("\[(.*?)\]", edges_raw[-2]).group(1)

'a(0),a(0)'

In [16]:
edges_raw[-2], edges_raw[2]

('connected(a(2),[a(0),a(0)],[1,0]).\n', 'connected(a(975),[a(186)],[1]).\n')

In [17]:
re.search("a\((.*)\),\[a\((.*?)\),a\((.*?)\)\]", edges_raw[-2]).group(3)

'0'

In [18]:
re.search("a\((.*)\),\[a\((.*?)\)\]", edges_raw[2]).group(0)

'a(975),[a(186)]'

In [19]:
edges_raw[-1]

'connected(a(1),[a(0),a(0)],[1,0]).\n'

In [20]:
things = []

In [21]:
edges = []
for i in range(len(edges_raw)):
    
    # Binary Operators
    b_match = re.search("a\((.*)\),\[a\((.*?)\),a\((.*?)\)\]", edges_raw[i])
    u_match = re.search("a\((.*)\),\[a\((.*?)\)\]", edges_raw[i])
    
    end, start_one, start_two = -1, -1, -1
    
    if b_match:
        end, start_one, start_two = int(b_match.group(1))-1, int(b_match.group(2))-1, int(b_match.group(3))-1
    else:
        end, start_one = int(u_match.group(1))-1, int(u_match.group(2))-1
    things.append(start_one)
    things.append(start_two)
    # start_one --> end
    # start_two --> end
    if start_one == start_two and start_one!=-1:
        edges.append((int(start_one), int(end)))     
    else:
        if start_one!=-1:
            edges.append((int(start_one), int(end)))
        if start_two!=-1:
            edges.append((int(start_two), int(end)))

In [22]:
len(set(things))

228

In [23]:
# func(f(872),[type=complex,def=(train(_137465):-has_car(_137465,_137471),wheels(_137471,3),has_car(_137465,_137483),long(_137483),has_car(_137465,_137494),wheels(_137494,3),has_car(_137465,_137506),long(_137506))]).
# func(f(50),[type=complex,def=(train(_137465):-has_car(_137465,_137471),wheels(_137471,3),has_car(_137465,_137483),long(_137483))]).

In [24]:
with open("data/sample5/edges.txt", "w") as fp:
    for u, v in edges:
        fp.write(f"{u} {v}\n")

In [25]:
len(edges), len(set(edges))

(1421, 1421)

In [26]:
num_neurons = max([max(u, v) for u, v in edges])+1

In [27]:
num_neurons

977

In [28]:
X_train = []
y_train = []

In [29]:
f = open("data/sample5/raw/rand_crm_1np_train_features_pos")

In [30]:
while True:
    gg = f.readline().split(" ")[3:-1]
    if not gg:
        break
    all_pos = [int(e)-1 for e in gg]
    dd = {i: 1 if i in all_pos else 0 for i in range(977)}
    X_train.append(dd)
    y_train.append(torch.tensor(1))

In [31]:
f = open("data/sample5/raw/rand_crm_1np_train_features_neg")

In [32]:
while True:
    gg = f.readline().split(" ")[3:-1]
    if not gg:
        break
    all_pos = [int(e)-1 for e in gg]
    dd = {i: 1 if i in all_pos else 0 for i in range(977)}
    X_train.append(dd)
    y_train.append(torch.tensor(0))

In [33]:
c = 0
for elem in X_train:
    c+=elem[326]
c

405

In [34]:
n.neurons[326].predeccesor_neurons, n.neurons[327].predeccesor_neurons, n.neurons[328].predeccesor_neurons

NameError: name 'n' is not defined

In [38]:
X_test = []
y_test = []

In [39]:
# !cat data/sample1/raw/rand_crm_1_test_features_pos

In [40]:
f = open("data/sample5/raw/rand_crm_1np_test_features_pos")

In [41]:
while True:
    gg = f.readline().split(" ")[3:-1]
    if not gg:
        break
    all_pos = [int(e)-1 for e in gg]
    dd = {i: 1 if i in all_pos else 0 for i in range(977)}
    X_test.append(dd)
    y_test.append(torch.tensor(1))

In [42]:
f = open("data/sample5/raw/rand_crm_1np_test_features_neg")

In [43]:
while True:
    gg = f.readline().split(" ")[3:-1]
    if not gg:
        break
    all_pos = [int(e)-1 for e in gg]
    dd = {i: 1 if i in all_pos else 0 for i in range(977)}
    X_test.append(dd)
    y_test.append(torch.tensor(0))

In [44]:
y_train[0]

tensor(1)

In [45]:
num_neurons = max([max(u, v) for u, v in edges])+1

In [46]:
num_neurons

977

In [47]:
adj_list = [[] for i in range(num_neurons)]

In [48]:
len(adj_list)

977

In [49]:
for u, v in edges:
    adj_list[u].append(v)

In [50]:
n = Network(num_neurons, adj_list)

In [51]:
orig_output_neurons = n.output_neurons

In [52]:
len(orig_output_neurons)

750

In [53]:
adj_list.append([])
adj_list.append([])

In [54]:
num_neurons = len(adj_list)

In [55]:
num_neurons

979

In [56]:
for i in range(num_neurons):
    if i in orig_output_neurons:
        adj_list[i].append(num_neurons-2)
        adj_list[i].append(num_neurons-1)

In [57]:
adj_list[842]

[977, 978]

In [58]:
n = Network(num_neurons, adj_list)

In [59]:
n.set_neuron_activation(977, lambda x : x, lambda x : 1)

In [60]:
n.set_neuron_activation(978, lambda x : x, lambda x : 1)

In [61]:
F.cross_entropy

<function torch.nn.functional.cross_entropy(input: torch.Tensor, target: torch.Tensor, weight: Union[torch.Tensor, NoneType] = None, size_average: Union[bool, NoneType] = None, ignore_index: int = -100, reduce: Union[bool, NoneType] = None, reduction: str = 'mean', label_smoothing: float = 0.0) -> torch.Tensor>

In [62]:
n.reset()
# X_train[2] = {i: 1 for i in range(len(X_train[2])+2)}
X_train[145][977]=1
X_train[145][978]=1
n.forward(X_train[145])

tensor([[52.7866],
        [48.6144]], grad_fn=<StackBackward0>)

In [63]:
# X_train[2]

In [64]:
sum([n.neurons[i].value for i in range(n.num_neurons)])

tensor([261.6547], grad_fn=<AddBackward0>)

In [65]:
n = Network(2, [[1], []])
n.forward({0:1, 1:1})

tensor([[0.7781]], grad_fn=<StackBackward0>)

In [66]:
len(adj_list)

979

In [67]:
n = Network(979, adj_list)

In [68]:
ip = {i: 1 for i in range(979)}
n.forward(ip)

tensor([[251.4023],
        [259.2228]], grad_fn=<StackBackward0>)

In [69]:
n.neurons[12].value

tensor(1)

In [70]:
n.neurons[6]

6: 1	 Grad: 0
            
Predecessor: []	Successor: [960, 887, 832, 781, 752, 722, 667, 264, 202, 198, 166, 149, 142, 139, 132, 123, 85, 82, 78, 64, 62, 47, 32]

In [71]:
n.neurons[970]

970: tensor([1.0957], grad_fn=<MulBackward0>)	 Grad: 0
            
Predecessor: [13, 136]	Successor: [977, 978]

In [72]:
n.neurons[59]

59: tensor([1.1884], grad_fn=<MulBackward0>)	 Grad: 0
            
Predecessor: [14, 15]	Successor: [826, 462, 429, 378]

In [73]:
n.neurons[327]

327: tensor([0.6368], grad_fn=<MulBackward0>)	 Grad: 0
            
Predecessor: [121]	Successor: [977, 978]

In [74]:
X_train[0][202]

0

In [75]:
for i in range(len(X_train)):
    X_train[i][977]=1
    X_train[i][978]=1

In [76]:
for i in range(len(X_test)):
    X_test[i][977]=1
    X_test[i][978]=1

In [77]:
assert len(X_train)==len(y_train)

In [78]:
assert len(X_test)==len(y_test)

In [79]:
y_train[0]

tensor(1)

In [80]:
save_object(adj_list, "data/sample5/adj_list.dill")

In [81]:
save_object(X_train, "data/sample5/X_train.dill")
save_object(X_test, "data/sample5/X_test.dill")
save_object(y_train, "data/sample5/y_train.dill")
save_object(y_test, "data/sample5/y_test.dill")